<a href="https://colab.research.google.com/github/nerudxlf/scientific_activity_on_wos_and_scopus/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pprint import pprint

from google.colab import files
files.upload()

In [ ]:
df_wos_citation = pd.read_excel("WoS.xls")
df_wos_data = pd.read_excel("wos5.xlsx")
df_oecd = pd.read_excel("OECD Category Mapping.xlsx")
list_sum: list = []
get_result_oecd_and_wos_category: list = []
get_list_all_wos_category: list = []
category_value_list: list = []
only_category_list: list = []
result_dict_citation: dict = {}

df_wos_citation.filter(["Название", "Всего цитат"])
df_wos_data.filter(["Article Title", "WoS Categories"])
join_df_by_title = pd.merge(left=df_wos_data, right=df_wos_citation, left_on="Article Title", right_on="Название")

list_citation = join_df_by_title["Всего цитат"].to_list()
list_research_areas = join_df_by_title["WoS Categories"].to_list()
list_wos_description = df_oecd['WoS_Description'].tolist()  # перевожу данные к списку
list_description = df_oecd["Description"].to_list()
uniq_dict_description = dict.fromkeys(list_description, "")

for i in range(len(list_research_areas)):
    if result_dict_citation.get(list_research_areas[i]):
        result_dict_citation[list_research_areas[i]] += list_citation[i]
    else:
        result_dict_citation |= {list_research_areas[i]: list_citation[i]}

for i in range(len(list_description)):
    uniq_dict_description[list_description[i]] += list_wos_description[i] + ";"

for key_c, item_c in result_dict_citation.items():
    dict_append = {}
    for key, item in uniq_dict_description.items():
        for i in key_c.split("; "):
            if item.find(i.upper()+";") != -1:
                dict_append |= {key: i.upper() + ":" + str(item_c)}
    get_result_oecd_and_wos_category.append(dict_append)
pprint(get_result_oecd_and_wos_category)

for element in get_result_oecd_and_wos_category:
    append_list = []
    for item in element.values():
        append_list.append(item)
    get_list_all_wos_category.append(list(set(append_list)))

for item in get_list_all_wos_category:
    for values in item:
        category_value_list.append(values)
        only_category_list.append(values.split(":")[0])
result_dict_category_value = dict.fromkeys(only_category_list, 0)

for elem in category_value_list:
    elem_split = elem.split(":")
    list_sum.append(int(elem_split[1]))
    result_dict_category_value[elem_split[0]] += int(elem_split[1])

list_to_df_title = list(result_dict_category_value.keys())
list_to_df_category = list(result_dict_category_value.values())
title_category_df = pd.DataFrame({"wos_category": list_to_df_title, "value": list_to_df_category})
result_df = pd.merge(left=df_oecd, right=title_category_df, right_on="wos_category",
                         left_on="WoS_Description")
result_df.to_excel("result_new.xlsx", index=False)
